<a href="https://colab.research.google.com/github/Bayhaqieee/SearchSort_OlistDatasets/blob/main/SearchSort_Olist_Team_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Binary Search - Counting Sort

In [ ]:
import pandas as pd
import time
from tabulate import tabulate

# Load dataset
GEO_URL = '/content/drive/MyDrive/Kuliah/Analgo/Dataset/olist_geolocation_dataset.csv'
PROD_URL = '/content/drive/MyDrive/Kuliah/Analgo/Dataset/olist_products_dataset.csv'

# Load the dataframes
geo_df = pd.read_csv(GEO_URL)
prod_df = pd.read_csv(PROD_URL)


# Kolom yang digunakan
geo_columns = {
    "Dataset": "Geolocation Dataset",
    "ID": "geolocation_zip_code_prefix",
    "Name": "geolocation_city",
    "Quantity": "geolocation_lat"
}

prod_columns = {
    "Dataset": "Products Dataset",
    "ID": "product_id",
    "Name": "product_category_name",
    "Quantity": "product_length_cm"
}

# Binary Search
def benchmark_binary_search(df, column, target):
    df_sorted = df.sort_values(by=column)
    data = df_sorted[column].dropna().values
    start = time.time()
    low, high = 0, len(data) - 1
    while low <= high:
        mid = (low + high) // 2
        if str(data[mid]) == str(target):
            break
        elif str(data[mid]) < str(target):
            low = mid + 1
        else:
            high = mid - 1
    end = time.time()
    return end - start

# Benchmark sorting dan binary search
def run_detailed_benchmark(df, dataset_info):
    results = []
    for op, col in [("Sort by", "ID"), ("Sort by", "Name"), ("Sort by", "Quantity")]:
        try:
            start = time.time()
            df.sort_values(by=dataset_info[col], inplace=False)
            elapsed = round(time.time() - start, 6)
        except:
            elapsed = "-"
        results.append([dataset_info["Dataset"], dataset_info[col], f"{op} {col}", elapsed])

    for col in ["ID", "Name", "Quantity"]:
        try:
            target = df[dataset_info[col]].dropna().iloc[len(df)//2]
            elapsed = round(benchmark_binary_search(df, dataset_info[col], target), 6)
        except:
            elapsed = "-"
        results.append([dataset_info["Dataset"], dataset_info[col], f"Binary Search {col}", elapsed])

    return results

# Buat ringkasan akhir
def summarize_benchmark(df, dataset_info, benchmark_results):
    row_count = len(df)
    summary = {
        "Dataset": dataset_info["Dataset"],
        "Rows": row_count
    }

    for role in ["ID", "Name", "Quantity"]:
        original_col = dataset_info[role]
        summary[f"{role} Column"] = original_col
        summary[f"{role} Used As"] = role

        # Ambil waktu sort
        sort_key = f"Sort by {role}"
        sort_time = next((r[3] for r in benchmark_results if r[1] == original_col and r[2] == sort_key), 0)

        # Ambil waktu search
        search_key = f"Binary Search {role}"
        search_time = next((r[3] for r in benchmark_results if r[1] == original_col and r[2] == search_key), 0)

        # Simpan masing-masing waktu
        summary[sort_key] = sort_time
        summary[search_key] = search_time

        # Total waktu (sort + search)
        if isinstance(sort_time, (int, float)) and isinstance(search_time, (int, float)):
            total_time = round(sort_time + search_time, 6)
        else:
            total_time = "-"
        summary[f"Total {role}"] = total_time

    return summary

# Jalankan benchmark
geo_results = run_detailed_benchmark(geo_df, geo_columns)
prod_results = run_detailed_benchmark(prod_df, prod_columns)

# Buat summary tabel
geo_summary = summarize_benchmark(geo_df, geo_columns, geo_results)
prod_summary = summarize_benchmark(prod_df, prod_columns, prod_results)

# Gabungkan dan tampilkan
summary_df = pd.DataFrame([geo_summary, prod_summary])
print(tabulate(summary_df, headers='keys', tablefmt='grid'))

+----+---------------------+---------+-----------------------------+--------------+--------------+--------------------+------------+-----------------------+----------------+----------------+----------------------+--------------+-------------------+--------------------+--------------------+--------------------------+------------------+
|    | Dataset             |    Rows | ID Column                   | ID Used As   |   Sort by ID |   Binary Search ID |   Total ID | Name Column           | Name Used As   |   Sort by Name |   Binary Search Name |   Total Name | Quantity Column   | Quantity Used As   |   Sort by Quantity |   Binary Search Quantity |   Total Quantity |
+====+=====================+=========+=============================+==============+==============+====================+============+=======================+================+================+======================+==============+===================+====================+====================+==========================+=========

# Jump Search - Heap Sort

In [ ]:
import pandas as pd
import time
import math
from tabulate import tabulate
import heapq

# Load dataset
GEO_URL = '/content/drive/MyDrive/Kuliah/Analgo/Dataset/olist_geolocation_dataset.csv'
PROD_URL = '/content/drive/MyDrive/Kuliah/Analgo/Dataset/olist_products_dataset.csv'

# Kolom yang digunakan
geo_columns = {
    "Dataset": "Geolocation Dataset",
    "ID": "geolocation_zip_code_prefix",
    "Name": "geolocation_city",
    "Quantity": "geolocation_lat"
}

prod_columns = {
    "Dataset": "Products Dataset",
    "ID": "product_id",
    "Name": "product_category_name",
    "Quantity": "product_length_cm"
}

# Heap Sort
def benchmark_heap_sort(df, column):
    data = df[column].dropna().tolist()
    start = time.time()
    heapq.heapify(data)
    sorted_data = [heapq.heappop(data) for _ in range(len(data))]
    end = time.time()
    return end - start

# Jump Search
def jump_search(arr, x):
    n = len(arr)
    step = int(math.sqrt(n))
    prev = 0
    while prev < n and arr[min(step, n)-1] < x:
        prev = step
        step += int(math.sqrt(n))
        if prev >= n:
            return -1
    for i in range(prev, min(step, n)):
        if arr[i] == x:
            return i
    return -1

def benchmark_jump_search(df, column, target):
    data = sorted(df[column].dropna().astype(str).values)
    start = time.time()
    jump_search(data, str(target))
    end = time.time()
    return end - start

# Benchmark per dataset
def run_detailed_benchmark(df, dataset_info):
    results = []
    for col in ["ID", "Name", "Quantity"]:
        try:
            sort_time = benchmark_heap_sort(df, dataset_info[col])
        except:
            sort_time = "-"
        results.append([dataset_info["Dataset"], dataset_info[col], f"Heap Sort {col}", round(sort_time, 6)])

    for col in ["ID", "Name", "Quantity"]:
        try:
            target = df[dataset_info[col]].dropna().iloc[len(df)//2]
            search_time = benchmark_jump_search(df, dataset_info[col], target)
        except:
            search_time = "-"
        results.append([dataset_info["Dataset"], dataset_info[col], f"Jump Search {col}", round(search_time, 6)])

    return results

# Ringkasan benchmark
def summarize_benchmark(df, dataset_info, benchmark_results):
    row_count = len(df)
    summary = {
        "Dataset": dataset_info["Dataset"],
        "Rows": row_count
    }

    for role in ["ID", "Name", "Quantity"]:
        original_col = dataset_info[role]
        summary[f"{role} Column"] = original_col
        summary[f"{role} Used As"] = role

        # Ambil waktu heap sort
        sort_key = f"Heap Sort {role}"
        sort_time = next((r[3] for r in benchmark_results if r[1] == original_col and r[2] == sort_key), 0)

        # Ambil waktu jump search
        search_key = f"Jump Search {role}"
        search_time = next((r[3] for r in benchmark_results if r[1] == original_col and r[2] == search_key), 0)

        # Simpan masing-masing waktu
        summary[sort_key] = sort_time
        summary[search_key] = search_time

        # Hitung total waktu (sort + search)
        if isinstance(sort_time, (int, float)) and isinstance(search_time, (int, float)):
            total_time = round(sort_time + search_time, 6)
        else:
            total_time = "-"
        summary[f"Total {role}"] = total_time

    return summary

# Jalankan benchmark
geo_results = run_detailed_benchmark(geo_df, geo_columns)
prod_results = run_detailed_benchmark(prod_df, prod_columns)

# Tampilkan tabel hasil
geo_summary = summarize_benchmark(geo_df, geo_columns, geo_results)
prod_summary = summarize_benchmark(prod_df, prod_columns, prod_results)

summary_df = pd.DataFrame([geo_summary, prod_summary])
print(tabulate(summary_df, headers='keys', tablefmt='grid'))


+----+---------------------+---------+-----------------------------+--------------+----------------+------------------+------------+-----------------------+----------------+------------------+--------------------+--------------+-------------------+--------------------+----------------------+------------------------+------------------+
|    | Dataset             |    Rows | ID Column                   | ID Used As   |   Heap Sort ID |   Jump Search ID |   Total ID | Name Column           | Name Used As   |   Heap Sort Name |   Jump Search Name |   Total Name | Quantity Column   | Quantity Used As   |   Heap Sort Quantity |   Jump Search Quantity |   Total Quantity |
+====+=====================+=========+=============================+==============+================+==================+============+=======================+================+==================+====================+==============+===================+====================+======================+========================+=========

# Jump Sort - Merge Sort

In [ ]:
import pandas as pd
import time
import math
from tabulate import tabulate

# Load dataset
GEO_URL = '/content/drive/MyDrive/Kuliah/Analgo/Dataset/olist_geolocation_dataset.csv'
PROD_URL = '/content/drive/MyDrive/Kuliah/Analgo/Dataset/olist_products_dataset.csv'

# Kolom yang digunakan
geo_columns = {
    "Dataset": "Geolocation Dataset",
    "ID": "geolocation_zip_code_prefix",
    "Name": "geolocation_city",
    "Quantity": "geolocation_lat"
}

prod_columns = {
    "Dataset": "Products Dataset",
    "ID": "product_id",
    "Name": "product_category_name",
    "Quantity": "product_length_cm"
}

# Merge Sort
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    merged = []
    i = j = 0
    while i < len(left) and j < len(right):
        if str(left[i]) <= str(right[j]):
            merged.append(left[i])
            i += 1
        else:
            merged.append(right[j])
            j += 1
    merged += left[i:]
    merged += right[j:]
    return merged

def benchmark_merge_sort(df, column):
    data = df[column].dropna().tolist()
    start = time.time()
    merge_sort(data)
    end = time.time()
    return end - start

# Jump Search
def jump_search(arr, x):
    n = len(arr)
    step = int(math.sqrt(n))
    prev = 0
    while prev < n and arr[min(step, n)-1] < x:
        prev = step
        step += int(math.sqrt(n))
        if prev >= n:
            return -1
    for i in range(prev, min(step, n)):
        if arr[i] == x:
            return i
    return -1

def benchmark_jump_search(df, column, target):
    data = sorted(df[column].dropna().astype(str).values)
    start = time.time()
    jump_search(data, str(target))
    end = time.time()
    return end - start

# Benchmark runner
def run_detailed_benchmark(df, dataset_info):
    results = []
    for col in ["ID", "Name", "Quantity"]:
        try:
            sort_time = benchmark_merge_sort(df, dataset_info[col])
        except:
            sort_time = "-"
        results.append([
            dataset_info["Dataset"],
            dataset_info[col],
            f"Merge Sort {col}",
            f"{sort_time:.6f}" if isinstance(sort_time, float) else "-"
        ])

    for col in ["ID", "Name", "Quantity"]:
        try:
            target = df[dataset_info[col]].dropna().iloc[len(df) // 2]
            search_time = benchmark_jump_search(df, dataset_info[col], target)
        except:
            search_time = "-"
        results.append([
            dataset_info["Dataset"],
            dataset_info[col],
            f"Jump Search {col}",
            f"{search_time:.6f}" if isinstance(search_time, float) else "-"
        ])
    return results

# Ringkasan hasil benchmark
def summarize_benchmark(df, dataset_info, benchmark_results):
    row_count = len(df)
    summary = {
        "Dataset": dataset_info["Dataset"],
        "Rows": row_count
    }

    for role in ["ID", "Name", "Quantity"]:
        original_col = dataset_info[role]
        summary[f"{role} Column"] = original_col
        summary[f"{role} Used As"] = role

        sort_key = f"Merge Sort {role}"
        search_key = f"Jump Search {role}"

        sort_time = next((r[3] for r in benchmark_results if r[1] == original_col and r[2] == sort_key), "0")
        search_time = next((r[3] for r in benchmark_results if r[1] == original_col and r[2] == search_key), "0")

        try:
            sort_time = f"{float(sort_time):.6f}"
        except:
            sort_time = "-"
        try:
            search_time = f"{float(search_time):.6f}"
        except:
            search_time = "-"

        summary[sort_key] = sort_time
        summary[search_key] = search_time

        try:
            total_time = float(sort_time) + float(search_time)
            summary[f"Total {role}"] = f"{total_time:.6f}"
        except:
            summary[f"Total {role}"] = "-"

    return summary

# Jalankan benchmark
geo_results = run_detailed_benchmark(geo_df, geo_columns)
prod_results = run_detailed_benchmark(prod_df, prod_columns)

geo_summary = summarize_benchmark(geo_df, geo_columns, geo_results)
prod_summary = summarize_benchmark(prod_df, prod_columns, prod_results)

summary_df = pd.DataFrame([geo_summary, prod_summary])

# Pastikan semua waktu dalam format string desimal, bukan notasi ilmiah
for col in summary_df.columns:
    summary_df[col] = summary_df[col].apply(lambda x: f"{float(x):.6f}" if isinstance(x, float) else x)

# Tampilkan tabel
print(tabulate(summary_df, headers='keys', tablefmt='grid'))


+----+---------------------+---------+-----------------------------+--------------+-----------------+------------------+------------+-----------------------+----------------+-------------------+--------------------+--------------+-------------------+--------------------+-----------------------+------------------------+------------------+
|    | Dataset             |    Rows | ID Column                   | ID Used As   |   Merge Sort ID |   Jump Search ID |   Total ID | Name Column           | Name Used As   |   Merge Sort Name |   Jump Search Name |   Total Name | Quantity Column   | Quantity Used As   |   Merge Sort Quantity |   Jump Search Quantity |   Total Quantity |
+====+=====================+=========+=============================+==============+=================+==================+============+=======================+================+===================+====================+==============+===================+====================+=======================+========================+

# Hash Search - Selection Sort

In [ ]:
import pandas as pd
import time
from tabulate import tabulate

# Load datasets
GEO_URL = '/content/drive/MyDrive/Kuliah/Analgo/Dataset/olist_geolocation_dataset.csv'
PROD_URL = '/content/drive/MyDrive/Kuliah/Analgo/Dataset/olist_products_dataset.csv'

# Load the dataframes
geo_df = pd.read_csv(GEO_URL)
prod_df = pd.read_csv(PROD_URL)

# Auto sample function with 20,000 limit
def auto_sample(df, max_rows=20000):
    return df.sample(n=max_rows, random_state=42) if len(df) > max_rows else df.copy()

# Define columns used
geo_columns = {
    "Dataset": "Geolocation Dataset",
    "ID": "geolocation_zip_code_prefix",
    "Name": "geolocation_city",
    "Quantity": "geolocation_lat"
}

prod_columns = {
    "Dataset": "Products Dataset",
    "ID": "product_id",
    "Name": "product_category_name",
    "Quantity": "product_length_cm"
}

# Selection Sort
def selection_sort(arr):
    arr = arr.copy()
    for i in range(len(arr)):
        min_idx = i
        for j in range(i+1, len(arr)):
            if str(arr[j]) < str(arr[min_idx]):
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    return arr

def benchmark_selection_sort(df, column):
    data = df[column].dropna().tolist()
    start = time.time()
    selection_sort(data)
    end = time.time()
    return end - start

# Hash Search
def hash_search(data_dict, target):
    return data_dict.get(target, None)

def benchmark_hash_search(df, column, target):
    data = df[column].dropna().astype(str).tolist()
    hash_map = {val: i for i, val in enumerate(data)}
    start = time.time()
    hash_search(hash_map, str(target))
    end = time.time()
    return end - start

# Benchmark runner
def run_detailed_benchmark(df, dataset_info):
    df = auto_sample(df)  # apply sampling
    results = []
    for col in ["ID", "Name", "Quantity"]:
        try:
            sort_time = benchmark_selection_sort(df, dataset_info[col])
        except Exception as e: # Catch general exception to prevent crash, useful for debugging
            print(f"Error during Selection Sort for column {dataset_info[col]}: {e}")
            sort_time = "-"
        results.append([
            dataset_info["Dataset"],
            dataset_info[col],
            f"Selection Sort {col}",
            f"{sort_time:.6f}" if isinstance(sort_time, float) else "-"
        ])

    for col in ["ID", "Name", "Quantity"]:
        try:
            # Ensure target is sampled appropriately from the potentially sampled df
            target = df[dataset_info[col]].dropna().iloc[len(df) // 2]
            search_time = benchmark_hash_search(df, dataset_info[col], target)
        except Exception as e: # Catch general exception
            print(f"Error during Hash Search for column {dataset_info[col]}: {e}")
            search_time = "-"
        results.append([
            dataset_info["Dataset"],
            dataset_info[col],
            f"Hash Search {col}",
            f"{search_time:.6f}" if isinstance(search_time, float) else "-"
        ])
    return results

# Summarize benchmark
def summarize_benchmark(df, dataset_info, benchmark_results):
    row_count = len(df)
    summary = {
        "Dataset": dataset_info["Dataset"],
        "Rows": row_count
    }

    for role in ["ID", "Name", "Quantity"]:
        original_col = dataset_info[role]
        summary[f"{role} Column"] = original_col
        summary[f"{role} Used As"] = role

        sort_key = f"Selection Sort {role}"
        search_key = f"Hash Search {role}"

        sort_time = next((r[3] for r in benchmark_results if r[1] == original_col and r[2] == sort_key), "0")
        search_time = next((r[3] for r in benchmark_results if r[1] == original_col and r[2] == search_key), "0")

        try:
            sort_time = f"{float(sort_time):.6f}"
        except:
            sort_time = "-"
        try:
            search_time = f"{float(search_time):.6f}"
        except:
            search_time = "-"

        summary[sort_key] = sort_time
        summary[search_key] = search_time

        try:
            # Convert strings back to floats for calculation, handle "-"
            sort_time_float = float(sort_time) if sort_time != "-" else 0
            search_time_float = float(search_time) if search_time != "-" else 0
            total_time = sort_time_float + search_time_float

            # Check if either was "-" and set total to "-" if so
            if sort_time == "-" or search_time == "-":
                 summary[f"Total {role}"] = "-"
            else:
                 summary[f"Total {role}"] = f"{total_time:.6f}"
        except:
            summary[f"Total {role}"] = "-" # Fallback in case of unexpected issue

    return summary

# Run benchmarks
# Pass the potentially sampled dataframes to the summary functions as well
geo_sampled_df = auto_sample(geo_df)
prod_sampled_df = auto_sample(prod_df)

geo_results = run_detailed_benchmark(geo_df, geo_columns)
prod_results = run_detailed_benchmark(prod_df, prod_columns)

# Use the sampled dataframes for summarizing to match row count
geo_summary = summarize_benchmark(geo_sampled_df, geo_columns, geo_results)
prod_summary = summarize_benchmark(prod_sampled_df, prod_columns, prod_results)


summary_df = pd.DataFrame([geo_summary, prod_summary])

# Show table using tabulate
# No longer need to format here as it's done in summarize_benchmark
print(tabulate(summary_df, headers='keys', tablefmt='grid'))

+----+---------------------+--------+-----------------------------+--------------+---------------------+------------------+------------+-----------------------+----------------+-----------------------+--------------------+--------------+-------------------+--------------------+---------------------------+------------------------+------------------+
|    | Dataset             |   Rows | ID Column                   | ID Used As   |   Selection Sort ID |   Hash Search ID |   Total ID | Name Column           | Name Used As   |   Selection Sort Name |   Hash Search Name |   Total Name | Quantity Column   | Quantity Used As   |   Selection Sort Quantity |   Hash Search Quantity |   Total Quantity |
+====+=====================+========+=============================+==============+=====================+==================+============+=======================+================+=======================+====================+==============+===================+====================+====================